# Fluorescence per phase

This module allows a calculations for a second fluorescence channel, based on cells that have been binned into cell cycle phases. There is also an option to ignore the phase information.

In [2]:
import os
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from loguru import logger
from GEN_Utils import FileHandling

2019-08-04 20:51:33,061 GEN_Utils.FileHandling: [INFO ]  Import ok


### Set some sample-specific parameters

In [3]:
input_folder = 'python/gauss_models/normalised/'
output_folder = 'examples/python/phase_fluorescence/'
fluorescence_col = 'TPE'
plate_samples = ['TPE only', '1', '1.5', '2', '3', '4']*4

plate_cords = [f'{x}{y}' for x in string.ascii_uppercase[0:4]
               for y in range(1, 7)]

sample_map = dict(zip(plate_cords, plate_samples))

In [4]:
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
# Generate filelist

file_list = [filename for filename in os.listdir(input_folder)]

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Dezerae/examples/python/phase_fluorescence/'

### Collect important info into summary df, grouped according to phase

In [ ]:
sample_data = []

for filename in file_list:
    sample_name = os.path.splitext(filename)[0]
    raw_data = pd.read_csv(f'{input_folder}{filename}')
    raw_data.rename(columns={fluorescence_col: "fluorescence"}, inplace=True)
    fluo_data = raw_data.copy()[['phase', 'fluorescence']]
    fluo_data = fluo_data.groupby('phase').median().T
    fluo_data['sample'] = sample_name
    sample_data.append(fluo_data)

summary_df = pd.concat(sample_data).reset_index(drop=True)

In [ ]:
summary_df.head()

In [ ]:
summary_df['plate'] = summary_df['sample'].str[0]
summary_df['well'] = summary_df['sample'].str[1:]
summary_df['sample'] = summary_df['well'].map(sample_map)
summary_df.sort_values(['sample'], inplace=True)

In [ ]:
summary_df.head(10)

### Generate equivalent dataset, ignoring phase

In [ ]:
sample_data = {}

for filename in file_list:
    sample_name = os.path.splitext(filename)[0]
    raw_data = pd.read_csv(f'{input_folder}{filename}')
    raw_data.rename(columns={fluorescence_col: "fluorescence"}, inplace=True)
    fluo_data = raw_data.copy()['fluorescence']
    sample_data[sample_name] = fluo_data.median()

summary_df = pd.DataFrame.from_dict(sample_data, orient='index').reset_index()

summary_df.rename(columns={'index': 'sample',
                           0: 'med_fluorescence'}, inplace=True)

summary_df['plate'] = summary_df['sample'].str[0]
summary_df['well'] = summary_df['sample'].str[1:]
summary_df['sample'] = summary_df['well'].map(sample_map)
summary_df.sort_values(['plate', 'sample'], inplace=True)

In [ ]:
summary_df.head(10)